# Classification of wind gust events

This notebook enables classification of daily maximum wind gust events into specific classes of storm type. Following the approach of [Cook (2023)][1], the notebook will plot the time history of gust wind speed, direction, temperature and station pressure over a 2-hour window around each gust event. The user then selects the storm type from one of 7 options.


[1]: Cook, N. J., 2023: Automated classification of gust events in the contiguous USA. *Journal of Wind Engineering and Industrial Aerodynamics*, **234**, 105330, https://doi.org/10.1016/j.jweia.2023.105330.


In [10]:
%matplotlib inline

import os
import time
from functools import partial

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import Layout, Box, VBox
from IPython.display import display, clear_output
import seaborn as sns
sns.set_style('whitegrid')
import warnings
warnings.filterwarnings("ignore", 'SettingWithCopyWarning')

In [11]:
BASEDIR = r"X:\georisk\HaRIA_B_Wind\data\derived\obs\1-minute\events"
hqstations = pd.read_csv(os.path.join(BASEDIR, "hqstations.csv"), index_col="stnNum")
options = [(stnName, idx) for idx, stnName in hqstations['stnName'].items()]

In [12]:
def loadData(stnNum):
    fname = os.path.join(BASEDIR, "events", f"{stnNum:06d}.pkl")
    df = pd.read_pickle(fname)
    df.reset_index(inplace=True)
    df.set_index(['date'], inplace=True)
    return df

def plotEvent(pdf, fig):
    fig.clear()
    ax = fig.add_axes([0, 0, 1, 1])
    ax2 = ax.twinx()
    axt = ax.twinx()
    axp = ax.twinx()
    pdf.plot.line(x='tdiff', y='windgust', marker='o', ax=ax)
    pdf.plot.scatter(x='tdiff', y='winddir', ax=ax2, color='g', marker='o',)
    pdf.plot.line(x='tdiff', y='tempanom', color='r', ax=axt)
    pdf.plot.line(x='tdiff', y='stnpanom', color='purple', ax=axp)
    axt.spines['right'].set_position(("axes", 1.1))
    axt.set_ylabel(r"Temperature anomaly [$^o$C]")
    ax.set_ylabel("Gust wind speed [km/h]")
    axp.set_ylabel("Pressure anomaly [hPa]")
    axp.spines['right'].set_position(('axes', 1.25))
    gmin, gmax = ax.get_ylim()
    ax.set_ylim((0, max(gmax, 100)))
    ax2.set_ylim((0, 360))
    ax2.set_yticks(np.arange(0, 361, 90))
    ax.set_title(pdf.index[0])
    ax.grid(True)
    ax2.grid(False)
    axt.grid(False)
    axp.grid(False)
    return fig

class Plotter:
    def __init__(self, df, fig, initial=0):
        self.df = df
        self.fig = fig
        self.value = initial
        self.nevents = len(self.df.index.unique('date'))
        eventdate = self.df.index.unique('date')[self.value]
        plotEvent(self.df[self.df.index==eventdate], self.fig)
    def plot(self, amount=1):
        if self.value == self.nevents - 1:
            print("No more events")
            self.fig.clf()
            return
        self.value += amount
        print(f"Event #: {self.value} of {self.nevents}")
        eventdate = self.df.index.unique('date')[self.value]
        plotEvent(self.df[self.df.index==eventdate], self.fig)
        return self.fig
    def __iter__(self, sentinal=False):
        return iter(self.plot, sentinal)

def button_callback(w):
    #out.clear_output(wait=True)
    with out:
        eventdate = df.index.unique('date')[plotter.value]
        print(f"Date: {eventdate} | Storm type: {selector.value}")
        eventlist.append((stnselector.value, eventdate, selector.value))
        
def plotnext(plotter, w):
    out.clear_output(wait=True)
    with out:
        plotter.plot()
        display(plotter.fig)

In [13]:
#def dropdown_eventhandler(change, names):
#    global df
#    print(change.value)
#    print(change.new)
#    df = loadData(change.new)
stnselector = widgets.Dropdown(options=options, description='Station')

In [14]:
selector = widgets.ToggleButtons(
    options=["Unclassified", "Synoptic", "Storm-burst", "Front down", "Front up", "Thunderstorm", "Spike"],
    description="Storm type",
    disabled=False
)
assignbtn = widgets.Button(description="Assign storm type")
nextevent = widgets.Button(description="Next event")
out = widgets.Output()

eventlist = []


In [15]:
display(stnselector)

Dropdown(description='Station', options=(('BROOME AIRPORT', 3003), ('PORT HEDLAND AIRPORT', 4032), ('LEARMONTH…

In [79]:
df = None
df = loadData(stnselector.value)
nevents = len(df.index.unique('date'))
fig = plt.figure(figsize=(12, 6))
plotter = Plotter(df, fig)
assignbtn.on_click(button_callback)
nextevent.on_click(partial(plotnext, plotter))

display(selector, assignbtn, nextevent)
out.clear_output(wait=True)
with out:
    plt.show()

out



ToggleButtons(description='Storm type', index=5, options=('Unclassified', 'Synoptic', 'Storm-burst', 'Front do…

Button(description='Assign storm type', style=ButtonStyle())

Button(description='Next event', style=ButtonStyle())

Output(outputs=({'name': 'stdout', 'text': 'No more events\n', 'output_type': 'stream'}, {'output_type': 'disp…

In [80]:
pd.DataFrame(eventlist, columns=['stnNum', 'date', 'stormType'])

,stnNum,date,stormType
0,3003,2003-02-06,Front up
1,3003,2008-12-23,Synoptic
2,3003,2012-03-17,Storm-burst
3,3003,2017-02-23,Front down
4,3003,2017-12-27,Synoptic
...,...,...,...
642,85072,2001-01-15,Spike
643,85072,2001-01-25,Spike
644,85072,2001-01-29,Spike
645,85072,2001-02-08,Spike
